# **Data Preparation Techniques**

## **Step 1: Setup Python Packages**

In [ ]:
pip install pandas pyodbc scikit-learn

In [ ]:
import pandas as pd
import pyodbc
from sklearn.preprocessing import StandardScaler

## **Step 2:** **Connect to the Database**

In [ ]:
server = 'SQLEXPRESS'
database = 'Transactions'
driver= '{SQL Server}'
trusted= 'yes'

# Connection string
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection={trusted}'

# Connect to the database
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

## **Step 3:** **Reading the Data into a DataFrame**

In [ ]:
query = '''
SELECT transaction_id, customer_id, 
       CAST(date AS DATE) as date, 
       CAST(time AS TIME) as time, 
       product_name, category, quantity, price
FROM [Transactions].[dbo].[Transactions]
'''
df = pd.read_sql_query(query, conn)

In [6]:
df.head()

,transaction_id,customer_id,date,time,product_name,category,quantity,price
0,1,C1001,2023-01-01,09:12:34.0000000,Shirt,Apparel,2,29.990000
1,2,C1002,2023-01-01,10:25:17.0000000,Headphones,Electronics,1,59.990002
2,3,C1003,2023-01-02,13:45:52.0000000,Book,Books,3,12.990000
3,4,C1004,2023-01-03,16:38:21.0000000,Shoes,Apparel,1,49.990002
4,5,C1005,2023-01-03,17:55:43.0000000,TV,Electronics,1,399.989990


## **Step 4:** **Create a Single Datetime Column**

In [ ]:
df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str))
df.set_index('datetime', inplace=True)

In [8]:
df.head()

,transaction_id,customer_id,date,time,product_name,category,quantity,price
datetime,,,,,,,,
2023-01-01 09:12:34,1,C1001,2023-01-01,09:12:34.0000000,Shirt,Apparel,2,29.990000
2023-01-01 10:25:17,2,C1002,2023-01-01,10:25:17.0000000,Headphones,Electronics,1,59.990002
2023-01-02 13:45:52,3,C1003,2023-01-02,13:45:52.0000000,Book,Books,3,12.990000
2023-01-03 16:38:21,4,C1004,2023-01-03,16:38:21.0000000,Shoes,Apparel,1,49.990002
2023-01-03 17:55:43,5,C1005,2023-01-03,17:55:43.0000000,TV,Electronics,1,399.989990


## **Step 5:** **Preprocessing and Feature Engineering**

In [ ]:
# Creating lag features
df['price_lag1'] = df['price'].shift(1)

# Extracting date-time features
df['day_of_week'] = df.index.dayofweek
df['month'] = df.index.month

# Creating rolling window features
df['rolling_mean_7'] = df['price'].rolling(window=7).mean()
df['rolling_std_7'] = df['price'].rolling(window=7).std()

# Handling missing values
df.fillna(method='ffill', inplace=True)

## **Step 6:** **Splitting the Dataset**

In [ ]:
split_idx = int(len(df) * 0.8)
train, test = df[:split_idx], df[split_idx:]

## **Step 7:** **Standardization**

In [ ]:
features = ['price_lag1', 'rolling_mean_7', 'rolling_std_7', 'day_of_week', 'month']

scaler = StandardScaler()
train_scaled = scaler.fit_transform(train[features])
test_scaled = scaler.transform(test[features])